# Apple's MLX: Input/Positional Embeddings and Multi-head Self-Attention 

#### Framework created by Apple.

In this tutorial, we will walk you through the core concepts and functionalities of MLX, starting with the basics of tokenization and positional encoding. Whether you’re a beginner looking to get started with machine learning on Apple hardware or an experienced practitioner seeking to optimize your workflows, this book provides practical examples and step-by-step instructions to help you harness the full potential of MLX.

In [127]:
import numpy as np
import mlx.core as mx
import mlx.nn as nn
from nltk.tokenize import word_tokenize
from collections import defaultdict

In [128]:
# Step 1: Tokenize the sentence
sentence = "What are the advantages and disadvantages of using a unified memory architecture?"
tokens = word_tokenize(sentence.lower())
print("Tokens:", tokens)

Tokens: ['what', 'are', 'the', 'advantages', 'and', 'disadvantages', 'of', 'using', 'a', 'unified', 'memory', 'architecture', '?']


In [129]:
# Step 2: Create a simple vocabulary and convert tokens to indices
vocab = defaultdict(lambda: len(vocab))
indices = [vocab[token] for token in tokens]
print("Indices:", indices)

Indices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


In [130]:
# Step 3: Initialize the embedding layer
embedding_dim = 64
num_embeddings = len(vocab)
embedding_layer = nn.Embedding(num_embeddings, embedding_dim)

In [131]:
# Convert indices to MLX array and embed them
input_data = mx.array(indices)
embedded_tokens = embedding_layer(input_data)
print("Embedded Tokens:", embedded_tokens)

Embedded Tokens: array([[-0.044703, -0.0250553, 0.213533, ..., -0.1252, 0.279171, -0.0321185],
       [-0.0280211, 0.135701, -0.182754, ..., 0.0769782, 0.0877975, 0.109833],
       [-0.0382277, -0.0457622, 0.0334275, ..., -0.0309626, -0.228566, 0.217419],
       ...,
       [-0.107577, 0.0191069, 0.207287, ..., 0.0617455, -0.119649, -0.0756776],
       [-0.0589486, 0.0437169, -0.186359, ..., -0.0349201, -0.0430669, 0.049653],
       [0.0568032, -0.0073198, -0.122325, ..., 0.154259, -0.0785468, 0.0552094]], dtype=float32)


## RoPE (Rotary Positional Encoding)

RoPE applies a rotational transformation to the token embeddings based on their positions in the sequence. This transformation uses sinusoidal functions to create a set of rotation matrices that are applied to the embeddings. The result is a set of positionally encoded embeddings that carry rich relative positional information.

In transformer models, understanding the relative positions of tokens within a sequence is crucial for tasks that require contextual understanding, such as language modeling and translation.

#### Calculate sequence length

In [132]:
seq_len = embedded_tokens.shape[0]
print("Sequence Length:", seq_len)

Sequence Length: 13


#### Generate frequencies for the sinusoidal embeddings

In [133]:
inv_freq = 1.0 / (10000 ** (np.arange(0, embedding_dim, 2).astype(np.float32) / embedding_dim))
print("Inverse Frequencies:", inv_freq)

freqs = mx.array(np.outer(np.arange(seq_len), inv_freq).astype(np.float32))
print("Frequencies:", freqs)

Inverse Frequencies: [1.0000000e+00 7.4989420e-01 5.6234133e-01 4.2169651e-01 3.1622776e-01
 2.3713736e-01 1.7782794e-01 1.3335215e-01 1.0000000e-01 7.4989416e-02
 5.6234129e-02 4.2169649e-02 3.1622779e-02 2.3713736e-02 1.7782794e-02
 1.3335215e-02 9.9999998e-03 7.4989423e-03 5.6234132e-03 4.2169648e-03
 3.1622779e-03 2.3713738e-03 1.7782794e-03 1.3335214e-03 1.0000000e-03
 7.4989418e-04 5.6234130e-04 4.2169649e-04 3.1622779e-04 2.3713738e-04
 1.7782794e-04 1.3335215e-04]
Frequencies: array([[0, 0, 0, ..., 0, 0, 0],
       [1, 0.749894, 0.562341, ..., 0.000237137, 0.000177828, 0.000133352],
       [2, 1.49979, 1.12468, ..., 0.000474275, 0.000355656, 0.000266704],
       ...,
       [10, 7.49894, 5.62341, ..., 0.00237137, 0.00177828, 0.00133352],
       [11, 8.24884, 6.18575, ..., 0.00260851, 0.00195611, 0.00146687],
       [12, 8.99873, 6.7481, ..., 0.00284565, 0.00213394, 0.00160023]], dtype=float32)


#### Calculate cosine and sine of the frequencies

In [134]:
cos_pos = mx.cos(freqs)
sin_pos = mx.sin(freqs)
print("Cosine Positional Encoding:", cos_pos)
print("Sine Positional Encoding:", sin_pos)

Cosine Positional Encoding: array([[1, 1, 1, ..., 1, 1, 1],
       [0.540302, 0.731761, 0.846009, ..., 1, 1, 1],
       [-0.416147, 0.0709483, 0.431463, ..., 1, 1, 1],
       ...,
       [-0.839072, 0.347628, 0.790132, ..., 0.999997, 0.999998, 0.999999],
       [0.0044257, -0.384674, 0.995257, ..., 0.999997, 0.999998, 0.999999],
       [0.843854, -0.910606, 0.893862, ..., 0.999996, 0.999998, 0.999999]], dtype=float32)
Sine Positional Encoding: array([[0, 0, 0, ..., 0, 0, 0],
       [0.841471, 0.681561, 0.533168, ..., 0.000237137, 0.000177828, 0.000133352],
       [0.909297, 0.99748, 0.902131, ..., 0.000474275, 0.000355656, 0.000266704],
       ...,
       [-0.544021, 0.937633, -0.612937, ..., 0.00237137, 0.00177828, 0.00133352],
       [-0.99999, 0.923052, -0.0972765, ..., 0.00260851, 0.00195611, 0.00146687],
       [-0.536573, 0.413275, 0.448343, ..., 0.00284564, 0.00213393, 0.00160023]], dtype=float32)


#### Split embedded tokens into even and odd parts

 Splits the embedded tokens into even and odd parts.

In [135]:
x1 = embedded_tokens[:, ::2]
x2 = embedded_tokens[:, 1::2]
print("x1 (even):", x1.shape)
print("x2 (odd):", x2.shape)

x1 (even): (13, 32)
x2 (odd): (13, 32)


#### Apply rotational transformation

Applies the rotational transformation to the split parts.

In [136]:
x1_new = x1 * cos_pos - x2 * sin_pos
x2_new = x1 * sin_pos + x2 * cos_pos
print("Transformed x1:", x1_new.shape)
print("Transformed x2:", x2_new.shape)

Transformed x1: (13, 32)
Transformed x2: (13, 32)


#### Concatenate the new x1 and x2 back together

Concatenates the transformed parts back together to get the final positional encoded embeddings.

In [137]:
positional_encoded_embeddings = mx.concatenate([x1_new, x2_new], axis=-1)
print("Positional Encoded Embeddings:", positional_encoded_embeddings)

Positional Encoded Embeddings: array([[-0.044703, 0.213533, -0.120804, ..., 0.247067, -0.1252, -0.0321185],
       [-0.129328, -0.157782, -0.0411983, ..., 0.272166, 0.0769838, 0.109845],
       [0.0575198, 0.0206218, -0.0881349, ..., -0.0591859, -0.0309707, 0.217358],
       ...,
       [0.100659, 0.176573, -0.187692, ..., 0.201885, 0.0619683, -0.0758371],
       [0.0434556, 0.0926936, -0.161226, ..., -0.00339627, -0.0345849, 0.0495897],
       [0.044006, 0.0521202, 0.155811, ..., 0.228939, 0.154033, 0.0550836]], dtype=float32)


# Multi-Head Self-Attention

To enhance the model’s ability to capture different types of relationships, the transformer employs multi-head self-attention. This technique splits the Query, Key, and Value vectors into multiple smaller sub-vectors, each corresponding to a different attention head. The attention mechanism is applied independently to each head, and the results are concatenated and linearly transformed to produce the final output.


In [138]:
# Define the dimensions
num_heads = 8
head_dim = embedding_dim // num_heads

# Sample input embeddings after positional encoding (from previous section)
# For demonstration, we assume `positional_encoded_embeddings` is already defined
input_embeddings = positional_encoded_embeddings

# Add a batch dimension if missing
if len(input_embeddings.shape) == 2:
    input_embeddings = input_embeddings[np.newaxis, :, :]

print("Input Embeddings Shape:", input_embeddings.shape)

Input Embeddings Shape: (1, 13, 64)


#### Linear Transformations

This cell applies linear projections to the input embeddings to obtain the Query, Key, and Value matrices.

In [139]:
# Step 1: Linear Transformations
query_proj = nn.Linear(embedding_dim, embedding_dim, bias=False)
key_proj = nn.Linear(embedding_dim, embedding_dim, bias=False)
value_proj = nn.Linear(embedding_dim, embedding_dim, bias=False)

queries = query_proj(input_embeddings)
keys = key_proj(input_embeddings)
values = value_proj(input_embeddings)

print("Queries Shape:", queries.shape)
print("Keys Shape:", keys.shape)
print("Values Shape:", values.shape)

Queries Shape: (1, 13, 64)
Keys Shape: (1, 13, 64)
Values Shape: (1, 13, 64)


#### Split for Multi-Head Attention

This cell reshapes the Query, Key, and Value matrices to prepare them for multi-head attention. It then transposes these matrices to separate the attention heads and prints their shapes.

In [140]:
# Step 2: Split for Multi-Head Attention
batch_size, seq_length, _ = queries.shape

queries = queries.reshape(batch_size, seq_length, num_heads, head_dim).transpose(0, 2, 1, 3)
keys = keys.reshape(batch_size, seq_length, num_heads, head_dim).transpose(0, 2, 1, 3)
values = values.reshape(batch_size, seq_length, num_heads, head_dim).transpose(0, 2, 1, 3)

print("Split Queries Shape:", queries.shape)
print("Split Keys Shape:", keys.shape)
print("Split Values Shape:", values.shape)

Split Queries Shape: (1, 8, 13, 8)
Split Keys Shape: (1, 8, 13, 8)
Split Values Shape: (1, 8, 13, 8)


#### Scaled Dot-Product Attention

This cell calculates the attention scores by performing scaled dot-product attention. It then applies the softmax function to obtain normalized attention weights and computes the final attention output.

In [141]:
# Step 3: Scaled Dot-Product Attention
dk = head_dim
scores = mx.matmul(queries, keys.transpose(0, 1, 3, 2)) / mx.sqrt(mx.array([dk], dtype=queries.dtype))
attention_weights = nn.softmax(scores, axis=-1)
attention_output = mx.matmul(attention_weights, values)

print("Attention Weights Shape:", attention_weights.shape)
print("Attention Output Shape:", attention_output.shape)

Attention Weights Shape: (1, 8, 13, 13)
Attention Output Shape: (1, 8, 13, 8)


#### Combine Heads

This cell combines the output from all attention heads back into a single tensor. It reshapes the combined output to match the original embedding dimensions and prints the shape.

In [142]:
# Step 4: Combine Heads
combined_output = attention_output.transpose(0, 2, 1, 3).reshape(batch_size, seq_length, embedding_dim)
print("Combined Output Shape:", combined_output.shape)

Combined Output Shape: (1, 13, 64)


#### Final Linear Transformation

This cell applies a final linear transformation to the combined output from the attention heads.

In [143]:
# Step 5: Final Linear Transformation
output_proj = nn.Linear(embedding_dim, embedding_dim, bias=False)
final_output = output_proj(combined_output)
print("Final Output after Self-Attention Shape:", final_output.shape)

Final Output after Self-Attention Shape: (1, 13, 64)


### Conclusion

In this notebook, we explored the foundational components of the transformer model architecture using the MLX framework, specifically tailored for Apple Silicon. Here’s a summary of what we covered:

1.	Tokenization and Embedding:

	- We began by tokenizing an example sentence into individual tokens using NLTK.
	- Each token was then converted into a unique index using a simple vocabulary.
	- We utilized an embedding layer to convert these token indices into dense vectors, preparing them for 	   further processing.
2.	Rotary Position Embedding (RoPE):

	- We applied the RoPE technique to enhance the input embeddings with positional information.
	- RoPE uses a rotational transformation based on sinusoidal functions to incorporate relative positional data directly into the embeddings.
	
3. Self-Attention Mechanism:
	- Following positional encoding, we implemented the self-attention mechanism, a core component of transformer models.
		- This included:
		- Linear transformations to obtain Query, Key, and Value matrices from the input embeddings.
		- Splitting these matrices into multiple heads for multi-head attention.
		- Calculating attention scores through scaled dot-product attention and applying the softmax function to obtain normalized attention weights.
		- Combining the output from all attention heads and applying a final linear transformation to produce the final self-attention output.
